<hr>
#1. Konstruisati matricu H LDPC 

- Matrica H je kljucna komponenta u procesu kodiranja i dekodiranja Low-Density Parity-Check (LDPC) kodovima.
- Matrica H je niske gustine, tj retka a to znaci da u njoj vecina elementa je nula.
- Matrica H se naziva i parity-check matricom ili kontrolna matrica ako vazi H·x<sup>T</sup>= 0 gde je x kodna rec

#Konstruisanje:
- Ako znamo da nam je:
    - n=15 broj kolona
    - m=9 broj redova
    - wr=5 sirina bloka (kolona)
    - wc=3 sirina bloka (redova)
- Imamo 3 bloka redova
    - prvi blok popunjavamo tako sto jedinice stavljamo po glavnoj dijagonali prvog bloka matrice, tako da svaki red ima po wr jedinica
    - drugi i treci blok popunjavamo random postavljanjem jedinica, ali tako da bude jedna i samo jedna jedinica u jednoj koloni tog bloka

In [26]:
import numpy as np
import random
import itertools

random.seed(58,2018)

n = 15  # Ukupan broj kolona u matrici H
k = 6   
wr = 5  # Širina bloka (kolona) u prvom setu redova
wc = 3  # Broj redova u svakom setu

H = np.zeros((n - k, n), dtype=int)  # Inicijalizacija matrice H

# Popunjavanje prvog seta redova
for i in range(wc):
    start_col = i * wr
    end_col = start_col + 5
    H[i, start_col:end_col] = 1

# Popunjavanje drugog seta redova
for j in range(n):
    rbr = random.randint(wc, wc + 2)
    H[wc:wc * 2, j] = (rbr == np.arange(wc, wc * 2)).astype(int)

# Popunjavanje trećeg seta redova
for j in range(n):
    rbr = random.randint(wc * 2, wc * 2 + 2)
    H[wc * 2:wc * 3, j] = (rbr == np.arange(wc * 2, wc * 3)).astype(int)

print(H)



[[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 1 1 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 0]
 [1 0 0 1 1 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 1 0 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 0 1 0 1 0 0 1 0 1]
 [0 0 1 0 0 0 0 0 0 0 1 1 0 1 0]]


<hr>
#2.1. Generisanje tabele sindroma i korektora

- Na pocetku je potrebno generisati sve binarne kombinacije za n bitova to nam predstavlja sve korektore e.
- Zatim ih sortiramo prema broju jedinica da prvo idu sve kombinacije koje imaju jednu jedinicu, pa dve itd. Tako obezbedimo wH(e) minimalnu tezinu
- Racunamo sve sindrome pomocu matrice H i vektora korektora e po obrascu: s = H·e<sup>T</sup>
- Konacnu tabelu sindroma i korektora formiramo tako sto pronadjemo prvu pojavu sindroma s, i taj korektor e zabelezimo.

In [27]:
# Generisanje svih binarnih kombinacija za n bitova
binarne_kombinacije = list(itertools.product([0, 1], repeat=n))

# Sortiranje binarnih kombinacija prema ukupnom broju jedinica
binarne_kombinacije_str_sortirane = sorted(binarne_kombinacije, key=lambda x: sum(x))

# Konvertovanje binarnih kombinacija u numpy matricu
matrica_binarnih_kombinacija = np.array(binarne_kombinacije_str_sortirane)

# Izračunavanje rezultata matričnog proizvoda
rezultat = np.dot(H, np.array(matrica_binarnih_kombinacija).T)

# Primena modulo 2 na rezultat
rezultat_mod_2 = np.mod(rezultat, 2)
finalniRezultati = (np.array(rezultat_mod_2).T) # Cisto zbog lepseg prikaza

# Pronalaženje jedinstvenih elemenata i njihovih indeksa
jedinstveni_elementi, indeksi = np.unique(finalniRezultati, axis=0, return_index=True)

# Izdvajanje binarnih kombinacija na osnovu indeksa
izdvojene_kombinacije = [binarne_kombinacije_str_sortirane[i] for i in indeksi]

# Ispis rezultata
print("KOREKTOR:             SINDROM")
for indeks, (jedinstveni_element, izdvojena_kombinacija) in enumerate(zip(jedinstveni_elementi, izdvojene_kombinacije)):
    print(f"{jedinstveni_element} : {izdvojena_kombinacija}")



KOREKTOR:             SINDROM
[0 0 0 0 0 0 0 0 0] : (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0 0 0 0 0 0 0 1 1] : (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0)
[0 0 0 0 0 0 1 0 1] : (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0)
[0 0 0 0 0 0 1 1 0] : (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0)
[0 0 0 0 1 1 0 0 0] : (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1)
[0 0 0 0 1 1 0 1 1] : (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1)
[0 0 0 0 1 1 1 0 1] : (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1)
[0 0 0 0 1 1 1 1 0] : (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1)
[0 0 0 1 0 1 0 0 0] : (1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0 0 0 1 0 1 0 1 1] : (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0 0 0 1 0 1 1 0 1] : (0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0 0 0 1 0 1 1 1 0] : (0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0 0 0 1 1 0 0 0 0] : (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0)
[0 0 0 1 1 0 0 1 1] : (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0)
[0 0 0 1 1 0 1 0 1

<hr>

#2.2. Kodno rastojanje
- Kodno rastojanje d(C) je minimalni broj linearno zavisnih kolona matrice H.
- Matrice su linearno zavisne ako postoji neka nenula linearna kombinacija tih matrica koja je jednaka nuli.
- Da bi smo pronasli minimalni broj zavisnih kolona, potrebno je da izracunamo sumu redova za sve moguce kombinacije kolona.
- Rezultat tj. kodno rastojanje predstavlja minalni broj kolona koje su linerano zavisne tj rezultat sabiranja redova ima je 0.

In [28]:
import numpy as np

niz = np.array(binarne_kombinacije)
matrica = np.array(H)

# Pravimo praznu listu za nove matrice
nove_matrice = []
# Pravimo praznu listu za brojeve kolona
brojevi_kolona = []
broj = []

# Funkcija za računanje zbira redova
def calculate_row_sums(matrix):
    zbir_reda = [sum(red) % 2 for red in matrix]
    if(zbir_reda == [1, 1, 0, 1, 1, 0, 0, 0, 0]):
        broj.append(len(matrix[0]))
    return zbir_reda


# Za svaki član niza
for clan in niz:
    # Pravimo praznu listu za novu matricu
    nova_matrica = []
    # Za svaki indeks i vrednost člana
    for i, v in enumerate(clan):
        # Ako je vrednost 1
        if v == 1:
            # Uzimamo i-tu kolonu originalne matrice
            kolona = matrica[:, i]
            # Dodajemo tu kolonu u novu matricu
            nova_matrica.append(kolona)
    # Pretvaramo novu matricu u NumPy niz
    nova_matrica = np.array(nova_matrica)
    # Transponujemo novu matricu da bude u ispravnom obliku
    nova_matrica = nova_matrica.T
    # Dodajemo novu matricu u listu
    nove_matrice.append(nova_matrica)
    # Dodajemo broj kolona u listu
    brojevi_kolona.append(nova_matrica.shape[1] if nova_matrica.ndim == 2 else 1)

# Ispisujemo nove matrice, zbir redova, broj kolona i listu brojeva kolona
for i, (m, clan, broj_kolona) in enumerate(zip(nove_matrice, niz, brojevi_kolona)):
    # print(f"Matrica {i + 1} (za član niza {clan}):")
    # print(m)
    zbir_redova = calculate_row_sums(m)
    # print(f"Zbir redova: {zbir_redova}")
    # print(f"Broj kolona: {broj_kolona}")
    # print("----------")

kodno_rastojanje = min(broj)

print("Kodno rastojanje:", kodno_rastojanje)

Kodno rastojanje: 2


<hr>

#3. Gallager B algoritam
- Pocinje se sa inicijalizacijom vrednosti informacionih bitova x(0) koje se postavljaju na vrednosti primljenog vektora y.

Iterativni koraci:

- Svaki informacioni bit xj salje svoju vrednost svim bitovima ci.
- Svaki bit ci racuna sumu ω(k)i→j koja predstavlja zbir vrednosti svih bitova osim xj.
- Informacioni bit xj prima vrednosti ω(k)i→j od ostalih bitova.

Na osnovu primljenih vrednosti, informacioni bit xj ažurira svoju vrednost u sledecoj iteraciji koristeci "vecinsko glasanje". Azuriranje se vrsi na osnovu broja pristiglih nula i jedinica:

- Ako je broj pristiglih nula (broj_nula) za informacioni bit xj veći ili jednak od th0 * (n - 1), tada se vrednost xj postavlja na 0.
- Ako je broj pristiglih jedinica (broj_jedinica) za informacioni bit xj veci ili jednak th1 * (n - 1), tada se vrednost xj postavlja na 1.
- Ovde (n - 1) predstavlja broj bitova za svaki informacioni bit, a th0 i th1 odredjuju koliki procenat bitova koji mora podrzavati promenu vrednosti da bi doslo do azuriranja.

Krajnji rezultat algoritma je dekodirana rec x(k), koja bi trebalo da predstavlja ispravnu kodnu rec.

In [1]:
def gallager_b_algoritam(y, th0=0.5, th1=0.5, max_iteracija=100):
    n = len(y)
 
    # Inicijalizacija
    x = list(y)  # Početna vrednost informacionih bitova

    for iteracija in range(max_iteracija):
        # Iterativni koraci
        for j in range(n):
            # Korak 1: Svaki xj šalje svoju vrednost ostalim bitovima
            poslate_vrednosti = [x[i] for i in range(n) if i != j]

            # Korak 2: Računanje sume ωi→j
            omega_i_do_j = sum(poslate_vrednosti)

            # Korak 3: Slanje ωi→j susedima
            for i in range(n):
                if i != j:
                    # Korak 2a: Efikasnije računanje ωi→j
                    omega_i = sum(x) - x[j]
                    omega_i_do_j = omega_i

                    # Korak 3: Slanje ωi→j 
                    poslata_vrednost = omega_i_do_j
                    # Korak 4: Pristizanje vrednosti ωi→j
                    primljene_vrednosti = [poslata_vrednost for _ in range(n) if _ != j]
                    # Korak 5: Ažuriranje vrednosti xj većinskim glasanjem
                    broj_nula = primljene_vrednosti.count(0)
                    broj_jedinica = primljene_vrednosti.count(1)
                    if broj_nula >= th0 * (n - 1):
                        x[j] = 0
                    elif broj_jedinica >= th1 * (n - 1):
                        x[j] = 1
                    else:
                        x[j] = int(y[j])  # Ako pragovi nisu zadovoljeni, zadržavamo vrednost iz kanala

        # Provera uslova zaustavljanja
        if all(x[i] == int(y[i]) for i in range(n)):
            print(f"Algoritam je dostigao stacionarno stanje posle {iteracija + 1} iteracija.")
            break

    return x

# Primer
primljeni_vektor = [1, 0, 0, 0, 0, 0, 0]
dekodirani_rezultat = gallager_b_algoritam(primljeni_vektor)
print("Dekodirani rezultat:", dekodirani_rezultat)


Dekodirani rezultat: [0, 0, 0, 0, 0, 0, 0]
